In [1]:
source extend_bashkernel-2modes.source

# This cell should be hidden and execute automatically when the page loads

# Make my-script the default when notebook is loaded:
set-global-evaluation-mode "my-script"

Ready.
Changing global mode to 'my-script'


To set the notebook mode, choose "my-script" or "provided-script" and run the cell.

In [ ]:
set-global-evaluation-mode "my-script"


# Jenkinsのセットアップ

## はじめに

今回は、Jenkinsを仮想マシンにインストールして演習を進めていきます。仮想マシンの操作にはIaaS基盤ソフトウェア「Wakame-vdc」を利用します。Wakame-vdcでは仮想マシンをインスタンスと呼んでいます。

## 演習について

* 本編の演習では、セルからコマンドを入力して実行しますが、Jenkinsが稼働するマシン上で実行する場合はsshを利用して実行して下さい。
  + ```IP="$(< ~/vdc_host_ip)" PORT="$(<~/vdc_instance_port)" ssh -i ~/mykeypair -p ~/vdc_instance_port root@$IP CommandLine```
* スクリプト例に記載されているコードは実際に実行できます。演習の参考にして下さい。
* また、スクリプト例は自動化スクリプトを記述する際の参考にもできます。

# Jenkinsをインストールする仮想マシンを起動する

## 準備

既に本編の演習を実施済みで、あらためて演習を実施する場合は、Wakame-vdcのGUIで稼働中のインスタンスをすべて破棄して下さい。
本編の演習を初めて実施する場合は必要ありません。

以下の手順でインスタンスを破棄します。
1. ブラウザでWakame-vdcのGUIにアクセスしてログインします。接続先については「接続情報」を参照して下さい。
* user: demo
* password: demo
2. [INSTANCES][Instances]をクリックします。「Instances」画面が表示されます。
3. 稼働しているインスタンスがリストされていますので、先頭のチェックボックスをすべてチェックします。
4. 画面上の[Setct an Action]プルダウンから[Terminate]を選択します。「Terminate Instances」ダイアログが開きます。
5. [Terminate]ボタンをクリックします。
6. 「Instances」画面に戻ります。インスタンスが破棄されると、Stateが"terminated"になります（Stateが変わらない場合は[Refesh]ボタンをクリックしてみて下さい）

<a href="https://cloud.githubusercontent.com/assets/9693694/13846358/d09c4b80-ec89-11e5-90da-c3b216619f7b.png"><img src="https://cloud.githubusercontent.com/assets/9693694/13846358/d09c4b80-ec89-11e5-90da-c3b216619f7b.png" width=30% height=30% align="left"></a>

### スクリプト例

Wakame-vdcには、Wakame-vdcの管理下にあるリソースを操作できるCUIが用意されています。
今回はGUI画面からインスタンスを破棄しましたが、次のスクリプトでもインスタンスが破棄できます。Jenkinsを使った自動化用途に利用できます。

In [ ]:
# Task: introduction-terminate-wakame-instances
# Evaluation mode: provided-script
# (if copying, only copy lines below here)

# Here is an automated alternative to using the Wakame-vdc GUI.
# The following Bash code will delete all Wakame-vdc instances by using
# the output from "mussel instance index" to get a list of instance IDs,
# and then using "mussel instance destroy" to terminate each ID.
# These mussel commands can also be used in an interactive terminal.

check_instances()
{
    mussel instance index | \
        (
            date=xxx # only instances database rows with empty "deleted_at" should be displayed
            while read ln; do
                case "$ln" in
                    *:id:*)
                        [ "$date" = "" ] && echo "$out"
                        read dash label theid <<<"$ln"
                        out="     $theid "
                        ;;
                    *:deleted_at:*)
                        read label date rest <<<"$ln"
                        ;;
                esac
            done
            [ "$date" = "" ] && echo "$out"
        )

}

ilist="$(check_instances 2>/dev/null)"
for inst in $ilist; do
    echo "Terminating instance: $inst"
    mussel instance destroy $inst 2>/dev/null
done

# This sleep is necessary so Wakame-vdc will reuse the 10.0.2.100 IP
# addresss.
sleep 15  # TODO: remove the need for this

# Finally, flush the arp cache to workaround what is probably a bug in
# the networking of the KVM that is hosting Jupyter.
# sudo ip -s -s neigh flush all


### 確認
次のセルを実行して、インスタンスがすべて破棄されていることを確認して下さい。

In [ ]:
# Task: introduction-terminate-wakame-instances
# Evaluation mode: load

# This cell should be hidden.

# copy/paste of the provided-script code:
check_instances()
{
    mussel instance index | \
        (
            date=xxx # only instances database rows with empty "deleted_at" should be displayed
            while read ln; do
                case "$ln" in
                    *:id:*)
                        [ "$date" = "" ] && echo "$out"
                        read dash label theid <<<"$ln"
                        out="     $theid "
                        ;;
                    *:deleted_at:*)
                        read label date rest <<<"$ln"
                        ;;
                esac
            done
            [ "$date" = "" ] && echo "$out"
        )

}

ilist="$(check_instances 2>/dev/null)"
for inst in $ilist; do
    echo "Terminating instance: $inst"
    mussel instance destroy $inst 2>/dev/null
done

# This sleep is necessary so Wakame-vdc will reuse the 10.0.2.100 IP
# addresss.
sleep 15  # TODO: remove the need for this


In [ ]:
# Task: introduction-terminate-wakame-instances
# Evaluation Mode: check

## 演習 : Wakame-vdcのインスタンスを起動する

### 演習の内容

まず、Jenkinsをインストールする仮想マシンを準備します。今回は、仮想マシンの操作にIaaS基盤ソフトウェア「Wakame-vdc」を利用して演習を進めていきます。Wakame-vdcでは仮想マシンをインスタンスと呼んでいます。<br>
それでは、Wakame-vdcのGUIにアクセスしてインスタンスを起動しましょう。<br>
以下の手順でインスタンスを起動します。
1. ブラウザからWakame-vdcのGUIにログインします。
* user: demo
* password: demo
2. 左メニューで[IMAGES][Machine Images]をクリックします。「Machine Images」画面が表示されます。
3. リストで「centos1d64」のラジオボタンをクリックします。
4. 画面上部の[Launch Instance]ボタンが有効になるのでクリックします。「Launch Instance」ダイアログが開きます。
5. ダイアログで以下のように設定します。
  * Instance Name: インスタンスを示す任意の名称を入力します。（例: jenkins）
  * HostName: 空欄のまま
  * Instance Spec: vz.xlarge
  * SSH Key Pair: ssh-cicddemo
  * Security Groups: cicddemo(sg-cicddemo)
  * Vif: eth0: [nw-demo1]demo1
6. [Launch]ボタンをクリックします。
7. [INSTANCES][Instances]をクリックします。「Instances」画面が表示されます。
8. 起動したインスタンスが表示されています。Stateが"running" になったらアクセス可能です。（"running"になる迄、若干時間がかかる場合があります）
  * <del>**IP欄に表示されていIPアドレスがJenkinsをインストールするマシンのIPアドレスです。**</del>

<a href="https://cloud.githubusercontent.com/assets/9693694/13846360/d3564506-ec89-11e5-843f-f3a71d77e76f.png"><img src="https://cloud.githubusercontent.com/assets/9693694/13846360/d3564506-ec89-11e5-843f-f3a71d77e76f.png" height="30%" width="30%" align="left"></a>
<a href="https://cloud.githubusercontent.com/assets/9693694/13846362/d7255e74-ec89-11e5-993a-682f15e14523.png"><img src="https://cloud.githubusercontent.com/assets/9693694/13846362/d7255e74-ec89-11e5-993a-682f15e14523.png" height="30%" width="30%" align="left"></a>
<a href="https://cloud.githubusercontent.com/assets/9693694/13846365/da32cf8e-ec89-11e5-9085-c9431a07362f.png"><img src="https://cloud.githubusercontent.com/assets/9693694/13846365/da32cf8e-ec89-11e5-9085-c9431a07362f.png" height="30%" width="30%" align="left"></a>

### スクリプト例

Wakame-vdcには、Wakame-vdcの管理下にあるリソースを操作できるCUIが用意されています。<br>
今回はGUI画面からインスタンスを起動しましたが、次のスクリプトでもインスタンスが起動できます。Jenkinsを使った自動化用途に利用できます。

In [ ]:
# Task: introduction-launch-wakame-instance
# Evaluation Mode: provided-script
# (if copying, only copy lines below here)


# Here is an automated alternative to using the Wakame-vdc GUI.
# The following Bash code will launch a Wakame-vdc instance by using
# "mussel instance create".

cat <<EOS >vifs.json
{
 "eth0":{"network":"nw-demo1","security_groups":"sg-cicddemo"}
}
EOS

mussel instance create --cpu-cores 2 --hypervisor openvz \
    --image-id wmi-centos1d64 --memory-size 2048 \
    --ssh-key-id ssh-cicddemo --vifs vifs.json --display-name centos

### 確認
次のセルを実行して、インスタンスの起動を確認して下さい。

In [ ]:
# Task: introduction-launch-wakame-instance
# Evaluation Mode: load

# This cell should be hidden.

cat <<EOS >vifs.json
{
 "eth0":{"network":"nw-demo1","security_groups":"sg-cicddemo"}
}
EOS

mussel instance create --cpu-cores 2 --hypervisor openvz \
    --image-id wmi-centos1d64 --memory-size 2048 \
    --ssh-key-id ssh-cicddemo --vifs vifs.json --display-name centos

In [ ]:
# Task: introduction-launch-wakame-instance
# Evaluation Mode: check

## 演習 : ssh接続を待つ

### 演習の内容

ここまでの演習で、インスタンスの起動をWakame-vdcのGUIで確認しました。ここではsshコマンドを使って、インスタンスが正常に起動していかを確認します。<br>
sshの実行に先立って、インスタンスのIPアドレスがWakame-vdcが割り当てたものであることを確認するようにして下さい。

**次のセルで、ssh接続を待つスクリプトを実行して下さい**

### スクリプト例

ssh接続を待つスクリプトの例です。

In [ ]:
# Task: introduction-wait-for-ssh
# Evaluation Mode: provided-script
# (if copying, only copy lines below here)


# The first part of this script waits for the ssh port (#22) to respond.
# At the end, an ssh connection is attempted.

IP="$(< ~/vdc_host_ip)"
PORT="$(<~/vdc_instance_port)"
while [[ "$(echo | nc -w 1 "$IP" "$PORT")" != *SSH* ]]; do
  sleep 2
  echo "Waiting on SSH at $IP..."
done
echo "SSH is active at $IP, port $PORT"

ssh -qi ~/mykeypair -p $PORT root@$IP uptime

### 確認
次のセルを実行して、ssh接続が可能であることを確認して下さい。

In [ ]:
# Task: introduction-wait-for-ssh
# Evaluation Mode: load

# This script should be hidden.

# It is also a bit awkward because the user's task is to check with ssh, but
# really waiting for ssh is necessary.  Should the user learn how to write
# a script that waits here?  If so how can it be checked?  ReRun the notebook.
# Hmmm, maybe that would be a nice demo.

IP="$(< ~/vdc_host_ip)"
PORT="$(<~/vdc_instance_port)"
while [[ "$(echo | nc -w 1 "$IP" "$PORT")" != *SSH* ]]; do
  sleep 2
  echo "Waiting on SSH at $IP..."
done
echo "SSH is active at $IP, port $PORT"

ssh -qi ~/mykeypair -p $PORT root@$IP uptime

In [ ]:
# Task: introduction-wait-for-ssh
# Evaluation Mode: check

# Jenkinsのインストール


## 演習 : java実行環境のインストール

### 演習の内容

javaの実行環境をインストールします。<br>
インストール後、/etc/profile.d/java.shも設定するようにして下さい。

**次のセルで、java実行環境のインストールとjava.shの設定スクリプトを実行して下さい。**

### スクリプト例

java実行環境をインストール後、java.shを設定するスクリプト例です。

In [ ]:
# Task: introduction-install-java
# Evaluation Mode: provided-script
# (if copying, only copy lines below here)

 

# This installs the Oracle version of Java.  The RPM file was
# already downloaded to ~/notebooks/downloads

IP="$(< ~/vdc_host_ip)"
PORT="$(<~/vdc_instance_port)"
cd ~/info
tar c jdk-8u73-linux-x64.rpm | ssh -qi ~/mykeypair -p $PORT root@$IP tar xv

ssh -qi ~/mykeypair -p $PORT root@$IP <<EOS

rpm -ivh jdk-8u73-linux-x64.rpm

cat <<CFG >/etc/profile.d/java.sh
JAVA_HOME=/usr/java/jdk1.8.0_25/
PATH=$JAVA_HOME/bin:$PATH
export PATH JAVA_HOME
export CLASSPATH=.
CFG

EOS

### 確認
次のセルを実行して、java実行環境のインストールと、java.shの設定が完了していることを確認して下さい。**

In [ ]:
# Task: introduction-install-java
# Evaluation Mode: check

## 演習 : Jenkins coreのインストール

### 演習の内容

Jenkins coreをインストールします。
インストールにはyumコマンドを使用します。バージョン「1.656-1.1」のJenkinsをインストールします。
```
yum install -y jenkins-1.656-1.1
```

**次のセルで、Jenkins coreをインストールして下さい。**

### スクリプト例

Jenkins coreインストールのスクリプト例です。

In [ ]:
# Task: introduction-install-jenkins
# Evaluation Mode: provided-script
# (if copying, only copy lines below here)

IP="$(< ~/vdc_host_ip)"
PORT="$(<~/vdc_instance_port)"
cd ~/info
tar c jenkins-1.656-1.1.noarch.rpm | ssh -qi ~/mykeypair -p $PORT root@$IP tar xv

ssh -qi ~/mykeypair -p $PORT root@$IP <<EOS

rpm -ivh jenkins-1.656-1.1.noarch.rpm

EOS

### 確認
次のセルを実行して、Jenkins coreがインストール済みであることを確認して下さい。

In [ ]:
# Task: introduction-install-jenkins
# Evaluation Mode: check

# 関連するツールのインストールとその他の設定

## 演習 : 関連するビルドツールのインストール

### 演習の内容

パッケージビルド等で必要なパッケージをインストールしておきます。次のパッケージが必要です。
* git iputils nc qemu-img parted kpartx rpm-build automake createrepo openssl-devel zlib-devel readline-devel gcc

**次のセルで、パッケージインストールを実行して下さい。**

### スクリプト例

パッケージインストールのスクリプト例です。

In [ ]:
# Task: introduction-install-others
# Evaluation Mode: provided-script
# (if copying, only copy lines below here)

IP="$(< ~/vdc_host_ip)"
PORT="$(<~/vdc_instance_port)"
ssh -qi ~/mykeypair -p $PORT root@$IP <<'EOS'

yum install -y git iputils nc qemu-img parted kpartx rpm-build automake createrepo openssl-devel zlib-devel readline-devel gcc

EOS

### 確認
次のセルを実行して、必要なパッケージがインストール済みであることを確認して下さい。

In [ ]:
# Task: introduction-install-others
# Evaluation Mode: check

## 演習 : sudo権限の付与

### 演習の内容

jenkinsアカウントがsudoを使える様に権限を付与します。本来は``visudo``コマンドを使用しますが、この演習ではリモートで設定しますので/etc/sudoersに直接追加します。

**次のセルで、sudo権限を付与するコマンドを実行して下さい。コマンド実行が成功しても、何も出力されません。**

### スクリプト例

sudo権限を付与するスクリプト例です。リモートで設定するため、/etc/sudoersに直接追加しています。

In [ ]:
# Task: introduction-sudo-config
# Evaluation Mode: provided-script
# (if copying, only copy lines below here)

IP="$(< ~/vdc_host_ip)"
PORT="$(<~/vdc_instance_port)"
ssh -qi ~/mykeypair -p $PORT root@$IP <<'EOS' 2>/dev/null

echo 'jenkins ALL=(ALL) NOPASSWD: ALL' >>/etc/sudoers

EOS

### 確認
次のセルを実行して、sudo権限が付与されていることを確認して下さい。

In [ ]:
# Task: introduction-sudo-config
# Evaluation Mode: check

## 演習 : tty設定の変更

### 演習の内容

sudoの初期設定ではttyを必要としています。Jenkinsからsudoを使うには、無効化する必要があるので無効化します。<br>
sedコマンドを使用して/etc/sudoersを編集します。
```
sed -i "s/^\(^Defaults\s*requiretty\).*/# \1/" /etc/sudoers
```

**次のセルで、tty設定を変更するコマンドを実行して下さい。コマンド実行が成功しても、何も出力されません。**

### スクリプト例

sudoersを編集するスクリプト例です。リモートで設定変更するため、/etc/sudoersを直接編集しています。

In [ ]:
# Task: introduction-sudo-notty
# Evaluation Mode: provided-script
# (if copying, only copy lines below here)

IP="$(< ~/vdc_host_ip)"
PORT="$(<~/vdc_instance_port)"
ssh -qi ~/mykeypair -p $PORT root@$IP <<'EOS' 2>/dev/null

sed -i "s/^\(^Defaults\s*requiretty\).*/# \1/" /etc/sudoers

EOS

### 確認
次のセルを実行して、tty設定が変更済みであることを確認して下さい。

In [ ]:
# Task: introduction-sudo-notty
# Evaluation Mode: check

# Jenkinsの起動

## 演習 : Jenkinを起動して起動完了を待つ

### 演習の内容

これまでの演習でJenkinsに関わるソフトウェアがインストールされ、Jenkinsが起動できる状態になりました。
それでは、Jenkinsを起動してみましょう。<br>
Jenkinsはserviceコマンドを使用して起動します。起動後、Jenkinsサービスが利用可能になるまで待つ処理も入れてみましょう。

**次のセルで、Jenkinsの起動とJenkinsサービスが利用可能になるまで待つ処理を記述して実行して下さい。**

### スクリプト例

In [ ]:
# Task: introduction-start-service
# Evaluation Mode: provided-script
# (if copying, only copy lines below here)

IP="$(< ~/vdc_host_ip)"
PORT="$(<~/vdc_instance_port)"

# (1) Enter script that uses ssh and service comands to
#     start the Jenkins service:
ssh -qi ~/mykeypair -p $PORT root@$IP <<'EOS'

service jenkins start

EOS

# (2) Next write a script here that waits for Jenkins 
#     to become ready:
ssh -qi ~/mykeypair -p $PORT root@${IP} <<'EOS'

while ! curl -I -s http://localhost:8080/ | grep -q "200 OK" ; do
    echo "Waiting for Jenkins..."
    sleep 3
done

echo "Jenkins is ready."

EOS

### 確認
次のセルを実行して、Jenkinsがサービスを開始していることを確認して下さい。

In [ ]:
# Task: introduction-start-service
# Evaluation Mode: check

# curlのSSL connect対応

curlのSSL接続対応のため、下記のセルを実行してlibcrulパッケージをアップデートして下さい。

In [ ]:
IP="$(< ~/vdc_host_ip)"
PORT="$(<~/vdc_instance_port)"

ssh -qi ~/mykeypair -p $PORT root@${IP} <<'EOS'
yum install -y epel-release
rpm -Uvh http://www.city-fan.org/ftp/contrib/yum-repo/city-fan.org-release-1-13.rhel6.noarch.rpm
sed -i "s,enabled=1,enabled=0," /etc/yum.repos.d/city-fan.org.repo
yum update -y --enablerepo=city-fan.org libcurl
EOS